In [1]:
# General packages

import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import json
import re


In [2]:
# Selenium

from selenium.webdriver.edge import service
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import Counter
from nltk.corpus import stopwords
from selenium.webdriver.common.action_chains import ActionChains

ModuleNotFoundError: No module named 'selenium'

# Collecting statistics on forum communications in each product page

In [3]:
# Load dataset of viable product pages:
pages = pd.read_csv("data/safe_obs_dataset.csv", sep=";", index_col = "Unnamed: 0")

In [4]:
pages

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_1,contrlink_2,contrlink_3,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,ext_links_ind
0,/t/app-6-0-compass-of-chance.4440211/,[APP][6.0+]Compass of Chance,"May 2, 2022","\nThe compass does not point to the north, but...",['https://play.google.com/store/apps/details?i...,Baiumka,/m/baiumka.12120441/,New member,"May 2, 2022",4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
2,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,"Apr 7, 2018",\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,https://forum.xda-developers.com/member.php?u=...,https://forum.xda-developers.com/member.php?u=...,https://forum.xda-developers.com/member.php?u=...,NaN,NaN,NaN,NaN,3.0,True,True
4,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,"Oct 25, 2015",\n\n\n\n\n\n\nNetGuard provides simple and adv...,['https://contact.faircode.eu/?product=netguar...,M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,https://forum.xda-developers.com/member.php?u=...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True
7,/t/app-2-2-root-wifi-reaver-gui-for-android.24...,[APP][2.2+][ROOT][WiFi] Reaver-GUI for Android,"Sep 24, 2013",\n\n\n\n\n\nReaver for Android v1.30\nReaver-W...,['https://bcmon.googlecode.com/files/bcmon.apk...,SOEDI,/m/soedi.4096618/,Senior Member,"May 14, 2011",154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
8,/t/app-11-dual-camera-video-recorder.4447581/,[APP][11+] Dual Camera Video Recorder,Today at 8:08 AM,\nCapture video from rear and front cameras at...,['https://play.google.com/store/apps/details?i...,User848,/m/user848.5214037/,Senior Member,"Apr 9, 2013",78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38396,/t/facebook-app-and-chat.678652/,Facebook app and chat,"May 9, 2010",\nI have try to find a real god app to faceboo...,[],tertix,/m/tertix.2493520/,New member,"Apr 11, 2010",4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38406,/t/free-apps-packet-good-apps-packet.678403/,[FREE APPS PACKET] Good apps packet.,"May 9, 2010",\neeehh... sory delete this topic\n \n,[],Cypek,/m/cypek.2556290/,New member,"May 6, 2010",3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38415,/t/full-video-chatt-app-like-skype.675381/,Full Video Chatt App like skype,"May 4, 2010",\nim deploying soon and was wondering if anyon...,[],pyrator,/m/pyrator.1603376/,Senior Member,"Feb 28, 2009",307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38435,/t/market-as-separate-app.672499/,Market as separate app,"Apr 29, 2010","\nHey people, I rebooted my phone this morning...",[],fuz1987,/m/fuz1987.2336665/,Member,"Feb 3, 2010",20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True


In [5]:
pages_dict = pages[["url", "title","main_developer_username"]].to_dict(orient="records")

In [6]:
pages_dict

[{'url': '/t/app-6-0-compass-of-chance.4440211/',
  'title': '[APP][6.0+]Compass of Chance',
  'main_developer_username': 'Baiumka'},
 {'url': '/t/app-all-root-solutions-6-0-viper4android-fx-2-7.3774651/',
  'title': '[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android FX 2.7',
  'main_developer_username': 'Team_DeWitt'},
 {'url': '/t/app-6-0-netguard-no-root-firewall.3233012/',
  'title': '[APP][6.0+] NetGuard - No-root firewall',
  'main_developer_username': 'M66B'},
 {'url': '/t/app-2-2-root-wifi-reaver-gui-for-android.2456888/',
  'title': '[APP][2.2+][ROOT][WiFi] Reaver-GUI for Android',
  'main_developer_username': 'SOEDI'},
 {'url': '/t/app-11-dual-camera-video-recorder.4447581/',
  'title': '[APP][11+] Dual Camera Video Recorder',
  'main_developer_username': 'User848'},
 {'url': '/t/app-root-nonroot-official-adaway-v5-12-1-v6-0-0b.2190753/',
  'title': '[APP][ROOT/NONROOT][OFFICIAL] AdAway v5.12.1/ v6.0.0b',
  'main_developer_username': 'PerfectSlayer'},
 {'url': '/t/app-magisk-modul

In [ ]:
# USE THE RIGHT DATA FOR THE SCRAPE!! DONT COMMIT BEFORE PULLING!
counter = 0

for page in pages_dict[0:2000]:    
    url = "https://forum.xda-developers.com" + page["url"]
    r = requests.get(url)
    if r:
        soup = BeautifulSoup(r.text)
        dev_comm = []
        consumer_comm = []
        dev_replies = []
        
        for message in soup.find_all(class_="message-inner"):
            post = {}
            post["text"] = message.find(class_="bbWrapper").get_text()
            post["date"] = message.find(class_="u-dt").get_text()
            post["author"] = message.find(class_="username").get_text()
            post["author_link"] = message.find("h4").a.get("href")
            post["author_title"] = message.find("h5").get_text()
            if re.search("Deleted member", str(soup.find(class_="message-userExtras"))) is None:
                if message.find(class_="username--style1") is None:
                    post["main_developer_joined"] = message.find(class_="message-userExtras").find_all("dd")[0].get_text()
                    post["main_developer_messages"] = message.find(class_="message-userExtras").find_all("dd")[1].get_text()
                    post["main_developer_reactions"] = message.find(class_="message-userExtras").find_all("dd")[2].get_text()

            if post["author"]==page["main_developer_username"]:
                dev_comm.append(post.copy())
                if message.find_all(class_="bbCodeBlock-sourceJump"):
                    if re.search(page["main_developer_username"], str(message.find(class_="bbCodeBlock-sourceJump").get_text())) is None:
                        dev_replies.append(post.copy())
                
            else:
                consumer_comm.append(post.copy())
                
        time.sleep(2)
        if soup.find_all(class_="pageNav-main"):
            page_number = int(soup.find(class_="pageNav-main").find_all("li")[4].get_text())+1
            for thread_page in range(2, page_number):
                url_next = url + 'page-'+str(thread_page)
                r_next = requests.get(url_next)
                time.sleep(2)
                if r_next:
                    soup = BeautifulSoup(r_next.text)
                    for message in soup.find_all(class_="message-inner"):
                        post = {}
                        post["text"] =  message.find(class_="bbWrapper").get_text()
                        post["date"] = message.find(class_="u-dt").get_text()
                        post["author"] = message.find(class_="username").get_text()
                        post["author_link"] = message.find("h4").a.get("href")
                        post["author_title"] = message.find("h5").get_text()
                        if re.search("Deleted member", str(soup.find(class_="message-userExtras"))) is None:
                            if message.find(class_="username--style1") is None:
                                post["main_developer_joined"] = message.find(class_="message-userExtras").find_all("dd")[0].get_text()
                                post["main_developer_messages"] = message.find(class_="message-userExtras").find_all("dd")[1].get_text()
                                post["main_developer_reactions"] = message.find(class_="message-userExtras").find_all("dd")[2].get_text()

                        if post["author"]==page["main_developer_username"]:
                            dev_comm.append(post.copy())
                            if message.find_all(class_="bbCodeBlock-sourceJump"):
                                if re.search(page["main_developer_username"], str(message.find(class_="bbCodeBlock-sourceJump").get_text())) is None:
                                    dev_replies.append(post.copy())
                        else:
                            consumer_comm.append(post.copy())
                
        page["consumer_comm"] = consumer_comm
        page["dev_comm"] = dev_comm
        page["dev_replies"] = dev_replies
        page["number_comm"] = len(consumer_comm)+len(dev_comm)
        page["number_devreplies"] = len(dev_replies)    
    
    else:
        print('Something is wrong!')
    
        counter += 1
        if counter % 100 == 0:
            print(counter)

In [11]:
url_next

'https://forum.xda-developers.com/t/app-all-root-solutions-6-0-viper4android-fx-2-7.3774651//page-47'

In [13]:
pd.DataFrame(pages_dict)

,url,title,main_developer_username,consumer_comm,dev_comm,dev_replies,number_comm,number_devreplies
0,/t/app-6-0-compass-of-chance.4440211/,[APP][6.0+]Compass of Chance,Baiumka,"[{'text': 'The idea sounds interesting, so may...",[{'text': 'The compass does not point to the n...,[],3.0,0.0
1,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,Team_DeWitt,NaN,NaN,NaN,NaN,NaN
2,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,M66B,NaN,NaN,NaN,NaN,NaN
3,/t/app-2-2-root-wifi-reaver-gui-for-android.24...,[APP][2.2+][ROOT][WiFi] Reaver-GUI for Android,SOEDI,NaN,NaN,NaN,NaN,NaN
4,/t/app-11-dual-camera-video-recorder.4447581/,[APP][11+] Dual Camera Video Recorder,User848,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
13369,/t/facebook-app-and-chat.678652/,Facebook app and chat,tertix,NaN,NaN,NaN,NaN,NaN
13370,/t/free-apps-packet-good-apps-packet.678403/,[FREE APPS PACKET] Good apps packet.,Cypek,NaN,NaN,NaN,NaN,NaN
13371,/t/full-video-chatt-app-like-skype.675381/,Full Video Chatt App like skype,pyrator,NaN,NaN,NaN,NaN,NaN
13372,/t/market-as-separate-app.672499/,Market as separate app,fuz1987,NaN,NaN,NaN,NaN,NaN


In [16]:
len(consumer_comm)

467

In [ ]:
message.find_all(class_="bbCodeBlock-content")

In [ ]:
pages_comm = pd.DataFrame(pages_dict)

In [ ]:
pages_comm.loc[2,"url"]

In [ ]:
page_number

In [ ]:
dev_comm = []
consumer_comm = []
dev_replies = []
for message in soup.find_all(class_="message-inner"):
    post{}
    post["text"] message.find_all(class_="bbCodeBlock-content"):
        if message.fin= message.find(class_="bbWrapper").get_text()
    post["date"] = message.find(class_="u-dt").get_text()
    post["author"] = message.find(class_="username").get_text()
    post["author_link"] = message.find("h4").a.get("href")
    post["author_title"] = message.find("h5").get_text()
    if re.search("Deleted member", str(soup.find(class_="message-userExtras"))) is None:
        if message.find(class_="username--style1") is None:
            post["main_developer_joined"] = message.find(class_="message-userExtras").find_all("dd")[0].get_text()
            post["main_developer_messages"] = message.find(class_="message-userExtras").find_all("dd")[1].get_text()
            post["main_developer_reactions"] = message.find(class_="message-userExtras").find_all("dd")[2].get_text()
    
    if post["author"]==page["main_developer_username"]:
        dev_comments.append(post)
        if message.find_all(class_="bbCodeBlock-content"):
            if re.search(page["main_developer_usernamce"], str(message.find(class_="bbCodeBlock-sourceJump").get_text())) is not:
                
    else:
        consumer_comm.append(post)
